This is the implementation of example V.C. of the paper *Variational approach to rare event simulation using least-squares regression, C. Hartmann, O. Kebiri, L. Neureither, L. Richter,
submitted to Chaos, 2019.*

We consider the metastable Langevin dynamics
$$d X_t = -\nabla U(X_t) dt + \sigma d W_t$$
with $U(x) = (x^2-1)^2$ and want to compute the probability
$$\psi(x, t) = P(\tau_O < T | X_t = x),$$
where $\tau_O = \inf \{t > 0 : X_t \notin O \}, O = (\infty, 0),$ by approximating the corresponding optimal control in an importance sampling attempt.

In [ ]:
%matplotlib inline

import copy
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy import exp, log, sqrt
from numpy.linalg import cond, inv, matrix_rank, norm
from scipy.io import loadmat

In [ ]:
def V(x):
    return (x**2 - 1)**2

def gradV(x):
    return 4 * x * (x**2 - 1)

def V_tilted(x):
    return (x**2 - 1)**2 - x

x_val = np.linspace(-2, 2, 100)
plt.plot(x_val, V(x_val));
plt.plot(x_val, V_tilted(x_val));

In [ ]:
sigma_g = 0.5    # standard deviation in ansatz function
M = 5    # amount of ansatz functions
initial_c = 0    # added constant drift in first forward iteration
X_0 = -1    # start value of SDE
dt = 0.001    # time discretization
T = 1    # terminal time
K = 1000    # number of trajectories
J = 5    # iterations of LSMC
N = int(np.floor(T / dt))   
epsilon = 0.01    # regularization parameter
sigma = 0.75    # noise of SDE

xs = [-0.5, -1]    # x values to plot

# load reference solution from MATLAB (done with pdepe)
psi = np.flip(loadmat('psi_sigma_0_75.m')['PROBAB'], 0)
gamma_eps = -np.log(psi + epsilon)
gamma_eps_selection = gamma_eps[:, 300:]

def gauss(x, m):
    return 1 / np.sqrt(2 * np.pi * sigma_g**2) * np.exp(-(x - m)**2 / (2 * sigma_g**2))

def dgauss(x, m):
    return -(x - m) / (np.sqrt(2 * np.pi) * sigma_g**3) * np.exp(-(x - m)**2 / (2 * sigma_g**2))

# this gives the possibilty to have basis funtions that move with time 
X_basis = np.repeat([np.linspace(-1.5, 0, M)], N, axis=0).T
basis = np.array([[gauss(x, np.array([phi])) for x in np.linspace(-3, 3, 100)] 
                  for phi in X_basis[:, 0]]) 

for m in range(M):
    plt.plot(np.linspace(-3, 3, 100), basis[m, :]);
    
fig = plt.figure(figsize=(10, 10))
ax = fig.gca(projection='3d')

x_span = np.linspace(-2, 0, 101)
t_span = np.linspace(0, 1, 101)
x_grid, t_grid = np.meshgrid(x_span, t_span)
surf = ax.plot_surface(x_grid, t_grid, gamma_eps_selection, linewidth=1, cmap=cm.coolwarm,
                       rstride=1, cstride=1)

In [ ]:
print('psi(x=-1,t=0) = %.2e' % psi[0, 350])

X = np.zeros([K, N + 1])
Y = np.zeros([K, N + 1])
Z = np.zeros([K, N + 1])

alpha = np.zeros([M, N + 1])
A = np.zeros([K, M]) 
A_grad = np.zeros([K, M])
b = np.zeros(K)
xi = np.random.randn(K, N)

# X_basis = np.zeros([K, N + 1])
# xi_basis = np.random.randn(K, N)

def v(x, c):
    return c

def v_t(X, n, alpha, X_basis):
    if sum(alpha[:, n]) == 0:
        return initial_c
    A_grad = np.zeros([len(X), M])
    for m in range(M):
        A_grad[:, m] = dgauss(X, X_basis[m, n])
    return -sigma * A_grad.dot(alpha[:, n])

def indicator(x):
    return (x >= 0) * 1.0

gamma_hat = {}
conditions = {}
alphas = []

for j in range(J):
    
    alpha_old = copy.deepcopy(alpha)
    conditions[str(j)] = []
    
    X[:, 0] = np.random.uniform(-2, -0.2, K)# X_0
    # X_basis[:, 0] = X_0

    for n in range(N):
        selection = X[:, n] >= 0
        X[selection, n] = 0
        X[selection, n + 1] = 0
        X[~selection, n + 1] = (X[~selection, n] + (- gradV(X[~selection, n])
                                + sigma * v_t(X[~selection, n], n, alpha_old, X_basis)) * dt
                                + np.sqrt(dt) * sigma * xi[~selection, n])
        #X _basis[:, n + 1] = (X_basis[:, n] + (- gradV(X_basis[:, n])
        #                      + v_t(X_basis[:, n], n, alpha_old, X_basis)) * dt
        #                      + np.sqrt(dt) * sigma * xi_basis[:, n])
        
    print('hit: %d / %d' % (sum(X[:, N] >= 0), K))
              
    Y[:, N] = -np.log(indicator(X[:, N]) + epsilon)
    
    for n in range(N - 1, 0, -1):
                          
        selection = X[:, n] < 0
        h = (0.5 * Z[:, n + 1]**2 
              + Z[:, n + 1] * v_t(X[:, n + 1], n + 1, alpha_old, X_basis)) * dt * selection                 
        b = Y[:, n + 1] - h
                          
        for m in range(M):
            A[:, m] = gauss(X[:, n], X_basis[m, n])
            A_grad[:, m] = dgauss(X[:, n], X_basis[m, n])
                          
        alpha[:, n] = inv(A.T.dot(A)).dot(A.T).dot(b) 
        # alpha[:, n] = np.linalg.pinv(A.T.dot(A)).dot(A.T).dot(b) 

        conditions[str(j)].append(cond(A.T.dot(A)))
        
        Y[:, n] = A.dot(alpha[:, n])     
        Z[:, n] = sigma * A_grad.dot(alpha[:, n])
        
    alpha[:, 0] = alpha[:, 1]
    Y[:, 0] = A.dot(alpha[:, 0])  
            
    print('j = %d: %.4f, %.4f, %.4f' % (j, Y[0, 0], np.mean(Y[~np.isnan(Y[:, 0]), 0]),
                                        np.median(Y[~np.isnan(Y[:, 0]), 0])))

    alphas.append(copy.deepcopy(alpha))
    for x in xs:
        gamma_hat[str(j) + str(x)] = [alpha[:, n].dot(np.array([gauss(x, np.array([phi]))
                                      for phi in X_basis[:, n]])) for n in range(1, N-1)]
    
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
for i, x in  enumerate(xs):
    ax[i].set_title(r'$x = %.1f, \sigma = %.2f, \epsilon = %.0e$' % (x, sigma, epsilon))
    for j in range(J):
        ax[i].plot(np.linspace(0, 1, len(gamma_hat[str(j) + str(x)])),
                   gamma_hat[str(j) + str(x)], label=r'approximation $i = %d$' % j);
    if x == -0.5:
        n_x = 75
    if x == -1:
        n_x = 50
    ax[i].plot(np.linspace(0, 1, 101), gamma_eps_selection[:, n_x], '--', color='red',
               label='true solution');
    ax[i].legend();

In [ ]:
alpha = alphas[-1]

A = np.zeros([201, M])
for m in range(M):
    A[:, m] = gauss(np.linspace(-2, 2, 201), X_basis[m, 0])
gamma_hat_grid = np.zeros([N, 201])
for n in range(N):
    gamma_hat_grid[n, :] = A.dot(alpha[:, n])
    
A = np.zeros([101, M])
for m in range(M):
    A[:, m] = dgauss(np.linspace(-2, 0, 101), X_basis[m, 0])
u_hat_grid = np.zeros([N, 101])
for n in range(N):
    u_hat_grid[n, :] = -sigma * A.dot(alpha[:, n])
    
print('true psi(x=-1, t=0) = %.2e' % psi[0, 350])
print('true psi(x=-1, t=0) = %.2e' % (exp(-gamma_hat_grid[0, 50]) - epsilon))

In [ ]:
plt.rcParams.update({'font.size': 11})

t = 0.50
n_1 = int(np.floor(t * N))
n_2 = int(np.floor(t * 100))

fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax.set_title(r'potentials at $t = %.2f, \sigma = %.2f$' % (t, sigma))
ax.plot(x_val, V(x_val), label='original potential');
ax.plot(np.linspace(-2, 0, len(gamma_hat_grid[n_1, :101])), sigma * gamma_hat_grid[n_1, :101] + V(np.linspace(-2, 0, len(gamma_hat_grid[n_1, :101]))), label='tilted');
ax.plot(np.linspace(-2, 0, len(gamma_eps_selection[n_2, :])), sigma * gamma_eps_selection[n_2, :] + V(np.linspace(-2, 0, len(gamma_eps_selection[n_2, :]))), '--', label='optimally tilted');
ax.legend()
ax.set_xlim(-2, 2)
ax.set_ylim(-0.2, 4.5);
ax.set_xlabel('x');
#fig.savefig('img/tilted_potentials.pdf')

In [ ]:
x = -1
n_1 = int(np.floor(t * N))
n_2 = int(np.floor(t * 100))
n_x = int(np.floor((x + 2) * 50))

gamma_hat_grid = np.zeros([N, 201])
u_hat_grid = np.zeros([N, 101])

fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax.set_title(r'value function at $x = %.2f, \sigma = %.2f$' % (x, sigma))

for i, alpha in enumerate(alphas[:3]):
    A = np.zeros([201, M])
    for m in range(M):
        A[:, m] = gauss(np.linspace(-2, 2, 201), X_basis[m, 0])
    for n in range(N):
        gamma_hat_grid[n, :] = A.dot(alpha[:, n])

    ax.plot(np.linspace(0, 1, len(gamma_hat_grid[:, n_x])), sigma * gamma_hat_grid[:, n_x], label=r'iteration $i = %d$' % (i + 1));

ax.plot(np.linspace(0, 1, len(gamma_eps_selection[:, n_x])), sigma * gamma_eps_selection[:, n_x], '--', label='true solution');
ax.legend()
ax.set_xlabel('t');
#fig.savefig('img/double_well_LSMC_iteration_sigma_0_75.pdf')

In [ ]:
plt.plot(np.linspace(0, T, N-1), conditions['0'][::-1]);
plt.yscale('log')
plt.xlabel('t');
plt.title(r'condition number of $A^T A$');

## Girsanov reweighting

In [ ]:
def get_x_index(x):
    return np.floor((x + 2) * 50).astype(int)

def u_t(x, n):
    if n >= N:
        n = N-1
    return u_hat_grid[n, get_x_index(x)]

G = 100000
X_g = np.zeros([G, 2])
X_n = np.zeros([G, 2])

ito_int = np.zeros(G)
riemann_int = np.zeros(G)

X_g[:, 0] = -1
X_n[:, 0] = -1

for n in range(N):
    xi = np.random.randn(G)
    selection_g = X_g[:, 0] >= 0
    selection_n = X_n[:, 0] >= 0
    X_g[selection_g, 0] = 0
    X_g[selection_g, 1] = 0
    X_n[selection_n, 0] = 0
    X_n[selection_n, 1] = 0
    ut = u_t(X_g[~selection_g, 0], n)
    X_g[~selection_g, 1] = X_g[~selection_g, 0] + (- gradV(X_g[~selection_g, 0]) + sigma * ut) * dt + np.sqrt(dt) * sigma * xi[~selection_g]
    X_n[~selection_n, 1] = X_n[~selection_n, 0] - gradV(X_n[~selection_n, 0]) * dt + np.sqrt(dt) * sigma * xi[~selection_n]
    X_g[:, 0] = X_g[:, 1]
    X_n[:, 0] = X_n[:, 1]
    ito_int[~selection_g] += ut * sqrt(dt) * xi[~selection_g]
    riemann_int[~selection_g] += ut**2 * dt

girsanov = exp(- ito_int - 1 / 2 * riemann_int)

IS_mean = np.mean((indicator(X_g[:, 1]) + epsilon) * girsanov) - epsilon
IS_var = np.var((indicator(X_g[:, 1]) + epsilon) * girsanov)
IS_rel_error = sqrt(IS_var) / IS_mean

naive_mean = np.mean((indicator(X_n[:, 1])))
naive_var = np.var((indicator(X_n[:, 1])))
naive_rel_error = sqrt(naive_var) / naive_mean

print('sigma = %.2f, delta_t = %.1e, K = %d' % (sigma, dt, K))
print('samples: %d' % G)
print('real mean: %.4e\n' % psi[0, 350])
print('IS hit: %d' % sum(X_g[:, 1] >= 0))
print('IS mean: %.4e' % IS_mean)
print('IS var: %.4e' % IS_var)
print('IS rel error: %.4e\n ' % IS_rel_error)
print('naive hit: %d' % sum(X_n[:, 1] >= 0))
print('naive mean: %.4e' % naive_mean)
print('naive var: %.4e' % naive_var)
print('naive rel error: %.4e' % naive_rel_error)

## compute probabilities with naive MC

In [ ]:
G = 500
t = 0.5
N_mc = int(np.ceil((T - t) / dt))

X_mc = np.zeros([G, N_mc + 1])

X_0s = np.linspace(-2, 0, 50)
psi_mc = []
    
for X_0 in X_0s:
    X_mc[:, 0] = X_0

    for n in range(N_mc):
        selection = X_mc[:, n] >= 0
        xi = np.random.randn(sum(~selection))
        X_mc[selection, n] = 0
        X_mc[selection, n + 1] = 0
        X_mc[~selection, n + 1] = X_mc[~selection, n] + (- gradV(X_mc[~selection, n])) * dt + np.sqrt(dt) * sigma * xi
        
    psi_mc.append(np.mean(X_mc[:, N_mc] == 0))

In [ ]:
t = 0.5
n_1 = int(np.ceil(t * gamma_hat_grid.shape[0]))
n_2 = int(np.ceil(t * gamma_eps_selection.shape[0]))

fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.set_title('value function for $t = %.1f, \sigma = %.2f$' % (t, sigma))
ax.plot(np.linspace(-2, 2, len(gamma_hat_grid[n_1, :])), gamma_hat_grid[n_1, :], label=r'LSMC $\hat{V}^{\epsilon}$');
ax.plot(np.linspace(-2, 0, len(gamma_eps_selection[n_2, :])), gamma_eps_selection[n_2, :], '--', label=r'true $V^{\epsilon}$', color='red');
ax.plot(X_0s, -log(np.array(psi_mc) + epsilon), label=r'naive $\hat{V}^{\epsilon}$');

ax.set_xlabel('x')
ax.legend();